In [1]:
import cv2 as cv
import pandas as pd
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import os
import shutil

### Crear una nueva carpeta con las todas las fotos y sus nuevas versionaes, separadas y clasificadas por none, piedra, papel, tijeras

In [14]:
os.mkdir('dataset')
os.mkdir('dataset\\none')
os.mkdir('dataset\\piedra')
os.mkdir('dataset\\papel')
os.mkdir('dataset\\tijeras')

In [13]:
import shutil
shutil.rmtree('dataset')

In [8]:
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LINEAR)
  return result

In [15]:

for root, dirs, files in os.walk(".", topdown=False):
    # print(root)
    if len(files) != 0:
        clave = ''
        if root.__contains__('none'):
            clave = 'none'
        if root.__contains__('tijeras') or root.__contains__('tijera'):
            clave = 'tijeras'
        if root.__contains__('papel'):
            clave = 'papel'
        if root.__contains__('piedra'):
            clave = 'piedra'
        if clave != '':
            for name in files:
                ruta = os.path.join(root, name)
                img = cv.imread(ruta)
                img = cv.resize(img,(512,512))

                rotated_180 = cv.rotate(img, cv.ROTATE_180)
                rotated_45 = rotate_image(img,45)
                rotated_315 = rotate_image(img,315)

                shutil.copy(ruta, f"dataset/{clave}/{name}")
                cv.imwrite(f"dataset/{clave}/rotated_180_{name}", rotated_180)
                cv.imwrite(f"dataset/{clave}/rotated_45_{name}", rotated_45)
                cv.imwrite(f"dataset/{clave}/rotated_315_{name}", rotated_315)

# Usar el modelo y aplicarlo a la camara

### 1º cargaremos el modelo

In [1]:
from pathlib import Path
import cv2 as cv
import pandas as pd
# from mediapipe import solutions
# from mediapipe.framework.formats import landmark_pb2
import numpy as np
import os
import shutil

current_path = Path.cwd()
file = 'gesture_recognizer.task'
model_path = os.path.join(current_path, file)

print(model_path)

c:\Users\oscar.torresrodrigue\GitHub\BigData-IA\piedra_papel_tijera\gesture_recognizer.task


In [4]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
import matplotlib.pyplot as plt
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import time
import cv2


# STEP 2: Create an HandLandmarker object.
BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode


base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.GestureRecognizerOptions(base_options=base_options, running_mode=vision.RunningMode.VIDEO,
                                       num_hands=2)
options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)

with GestureRecognizer.create_from_options(options) as recognizer:

# Abrir la camara
    cap = cv2.VideoCapture(0)

    current_timestamp_ms = time.time()

    while (cap.isOpened()):
        ret, frame = cap.read()

        if ret == True:
        
            # STEP 3: Load the input image.
            
            image_bgr = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_bgr)
            
            # STEP 4: Detect hand gesture from the input image.
            frame_timestamp_ms = round((time.time() - current_timestamp_ms) * 1000)

            gesture_recognition_result = recognizer.recognize_for_video(mp_image, frame_timestamp_ms)
    
            # STEP 5: Process the classification result. In this case, visualize it.
            
            top_gesture = gesture_recognition_result.gestures
            if len(top_gesture) > 0:
                print(top_gesture)
            
            mp_image = mp_image.numpy_view()

            hand_landmarks = gesture_recognition_result.hand_landmarks

            cv2.imshow('Result', cv2.cvtColor(mp_image, cv2.COLOR_BGR2RGB))

            if cv2.waitKey(10) & 0xFF == ord('q'): break
    cap.release()
    cv2.destroyAllWindows()

[[Category(index=-1, score=0.93277907371521, display_name='', category_name='')]]
[[Category(index=-1, score=0.9405354857444763, display_name='', category_name='')]]
[[Category(index=-1, score=0.9385672211647034, display_name='', category_name='')]]
[[Category(index=-1, score=0.9340015649795532, display_name='', category_name='')]]
[[Category(index=-1, score=0.9292316436767578, display_name='', category_name='')]]
[[Category(index=-1, score=0.9311591386795044, display_name='', category_name='')]]
[[Category(index=-1, score=0.9315068125724792, display_name='', category_name='')]]
[[Category(index=-1, score=0.9356778264045715, display_name='', category_name='')]]
[[Category(index=-1, score=0.936050534248352, display_name='', category_name='')]]
[[Category(index=-1, score=0.9390888214111328, display_name='', category_name='')]]
[[Category(index=-1, score=0.9438708424568176, display_name='', category_name='')]]
[[Category(index=-1, score=0.5036144256591797, display_name='', category_name='

: 